# stSME Comparison

This tutorial shows the stSME normalization effect between of two scenarios: 
- (1) normal (without stSME) 
- (2) stSME applied on raw gene counts

In this tutorial we use **Mouse Brain (Coronal)** Visium dataset from [10x genomics website](https://support.10xgenomics.com/spatial-gene-expression/datasets/1.0.0/V1_Adult_Mouse_Brain).

In [ ]:
import scanpy as sc
import stlearn as st
import pathlib as pathlib
import numpy as np
import random as random
import os as os

st.settings.set_figure_params(dpi=120)

# Make sure all the seeds are set
seed = 0
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")

In [ ]:
st.settings.datasetdir =  pathlib.Path.cwd().parent / "data"

In [ ]:
data = sc.datasets.visium_sge(sample_id="V1_Adult_Mouse_Brain")
data = st.convert_scanpy(data)

In [ ]:
# pre-processing for gene count table
st.pp.filter_genes(data, min_cells=1)
st.pp.normalize_total(data)
st.pp.log1p(data)

In [ ]:
# pre-processing for spot image
st.pp.tiling(data, out_path="tiling")

# this step uses deep learning model to extract high-level features from tile images
# may need few minutes to be completed
st.pp.extract_feature(data)

In [ ]:
# run PCA for gene expression data
st.em.run_pca(data, n_comps=50)

### (1) normal (without stSME) 

In [ ]:
data_normal = data.copy()

#### marker gene for CA3

In [ ]:
i = "Lhfpl1"
st.pl.gene_plot(data_normal, gene_symbols=i, size=3)

#### marker gene for DG

In [ ]:
i = "Pla2g2f"
st.pl.gene_plot(data_normal, gene_symbols=i, size=3)

### (2) stSME applied on raw gene counts

In [ ]:
data_SME = data.copy()
# apply stSME to normalise log transformed data
st.spatial.SME.SME_normalize(data_SME, use_data="raw")
data_SME.X = data_SME.obsm['raw_SME_normalized']
st.em.run_pca(data_SME, n_comps=50)

#### marker gene for CA3

In [ ]:
i = "Lhfpl1"
st.pl.gene_plot(data_SME, gene_symbols=i, size=3)

#### marker gene for DG

In [ ]:
i = "Pla2g2f"
st.pl.gene_plot(data_SME, gene_symbols=i, size=3)

<b>Tutorial by Xiao Tan</b>